In [10]:
#This program is going to update a spreadsheet with 4 share holdings.
#At the close of each day it will evaluate which share grew the most that day.
#It will then calculate how much money we have, given the current share price, at the open tomorrow.
#It will then add to the spreadsheet how much we should buy of the largest growing share
#This uses the terrible and completely amateurish startegy of selling everything we have, to buy the highest growing yesterday with all the money we have.

from lxml import html
import requests
import datetime
import numpy as np

#We define the following program that will tell us if the following day is a day that banks are open.

def TradeOpenTomorrow():
    #This program determines in banks are open to trade tomorrow in 2019.
    # It returns 0 if banks are closed tomorrow. And 1 if they are open.
    #It does this by checking if tomorrow is a weekend or a bank holiday.
    tomorrow =  datetime.date.today() + datetime.timedelta(days=1)
    d1 = tomorrow.strftime("%d/%m/%Y")
    wkday=tomorrow.weekday()
    BHol2019=["01/01/2019","17/03/2019","19/04/2019","22/04/2019","06/05/2019","27/05/2019","12/07/2019","26/08/2019","25/12/2019","26/12/2019"]
    if wkday in [5,6]:
        return 0
    elif d1 in BHol2019:
        return 0
    else:
        return 1



In [11]:
#We now collect all of the financial information from MorningStar that we need.

#First collect Barclays info
pageB = requests.get('http://tools.morningstar.co.uk/uk/stockreport/default.aspx?Site=uk&id=0P00007NZP&LanguageId=en-GB&SecurityToken=0P00007NZP]3]0]E0WWE$$ALL')
treeB = html.fromstring(pageB.content)
NameB=treeB.xpath('//span[@class="securityName"]/text()')[0]
priceTodayB = treeB.xpath('//span[@class="price"]/text()')[0]
priceLastCloseB=treeB.xpath('//td[@id="Col0LastClose"]/text()')[0]
        
#Now collect HSBC data
pageH = requests.get('http://tools.morningstar.co.uk/uk/stockreport/default.aspx?Site=uk&id=0P00007OFH&LanguageId=en-GB&SecurityToken=0P00007OFH]3]0]E0WWE$$ALL')
treeH = html.fromstring(pageH.content)
NameH=treeH.xpath('//span[@class="securityName"]/text()')[0]
priceTodayH = treeH.xpath('//span[@class="price"]/text()')[0]
priceLastCloseH=treeH.xpath('//td[@id="Col0LastClose"]/text()')[0]
        
#Now collect Lloyd's data
pageL = requests.get('http://tools.morningstar.co.uk/uk/stockreport/default.aspx?Site=uk&id=0P000090RG&LanguageId=en-GB&SecurityToken=0P000090RG]3]0]E0WWE$$ALL')
treeL = html.fromstring(pageL.content)
NameL=treeL.xpath('//span[@class="securityName"]/text()')[0]
priceTodayL = treeL.xpath('//span[@class="price"]/text()')[0]
priceLastCloseL=treeL.xpath('//td[@id="Col0LastClose"]/text()')[0]
        
#Now collect RBS data
pageR = requests.get('http://tools.morningstar.co.uk/uk/stockreport/default.aspx?Site=uk&id=0P000090MW&LanguageId=en-GB&SecurityToken=0P000090MW]3]0]E0WWE$$ALL')
treeR = html.fromstring(pageR.content)
NameR=treeR.xpath('//span[@class="securityName"]/text()')[0]
priceTodayR = treeR.xpath('//span[@class="price"]/text()')[0]
priceLastCloseR=treeR.xpath('//td[@id="Col0LastClose"]/text()')[0]
        
#Now that we have all the finanical info that we need. We calculate the percentage change in the last two closes
#And the maximum change
Bper=float(priceTodayB)/float(priceLastCloseB)
Hper=float(priceTodayH)/float(priceLastCloseH)
Lper=float(priceTodayL)/float(priceLastCloseL)
Rper=float(priceTodayR)/float(priceLastCloseR)
Per=[Bper,Hper,Lper,Rper]
A=max(Per)

In [12]:
#We now create a function that will tell us what shares to buy at market open the next day.
#The output of the function will be a list of 0's and the stock price of the largest grower in a vector of length 4.
#The companies have been ordered alphabetically and a 0 shall mean don't hold entirely that stock.
def Portfollio():
    #We first check if tomorrow is a trading day.
    if TradeOpenTomorrow()==0:
        return
    else:
        


        #We now test which share gave the greatest change in the last trading day.
        
        
        
        if A==Bper:
            I=[float(priceTodayB),0,0,0]
        if A==Hper:
            I=[0,float(priceTodayH),0,0]
        if A==Lper:
            I=[0,0,float(priceTodayL),0]
        if A==Rper:
            I=[0,0,0,float(priceTodayR)]
        return I

 

In [13]:
#We now initialize my sheet which contains the data of my previous portfollios in the form
#date held, B shares, H shares, L shares, R shares, Cash , Cash equiv

#This first block of code initalizes the code accessing the google sheet of historic trades and cash value
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

ss = client.open('TradingBig4Banks')
ws = ss.worksheet("Trading")

In [14]:
#We now define a function that will calculate how much money we have at the end of the day
def PortfollioValue():
    #This function returns the current value of all holdings.
    TotShare=np.array([float(priceTodayB),float(priceTodayH),float(priceTodayL),float(priceTodayR),1])
    col_len = len(ws.col_values(1))
    vect=np.array(ws.row_values(col_len)[1:6])
    Amount=[float(i) for i in vect]
    Cash=np.dot(TotShare,Amount)
    return Cash


In [15]:
#We now define a function that can tell us what portfollio we should buy at the start of business tomorrow.
def NewPortfollio():
    #We first check if we can trade tomorrow.
    if TradeOpenTomorrow()==1:
        #Now we check to see how much cash we have
        Cash=PortfollioValue()
        #We now calculate the new portfollio
        NP=[0,0,0,0,0]
        A=Portfollio()
        for i in range(len(A)):
            if A[i]!=0:
                NP[i]=Cash/A[i]
        return NP
            
        
        
        
    else:
        return "NA"
    


In [16]:
#We now combine these to update the spreadsheet

def SheetUpdate():
    #We first start by checking that tomorrow's values haven't already been caluclated. As this is a one trade per day strat.
    #Otherwise it will just add another row to the spreadsheet.
    col_len = len(ws.col_values(1))
    tomorrow =  datetime.date.today() + datetime.timedelta(days=1)
    tomorrow=tomorrow.strftime("%d/%m/%Y")
    if ws.cell(col_len,1).value!=tomorrow:
        A=NewPortfollio()
        ValueAtClose=PortfollioValue()
        #here we have calculated the portfollio that we should buy at the begining of trading the next day, and the value that 
        #We get from selling our current portfollio.
        if A!= "NA":
            B = [tomorrow]
            B.extend(A)
            B.extend([ValueAtClose])
            ws.insert_row(B, col_len+1)

    else:
        return print("Only one trade allowed per day")

In [17]:
SheetUpdate()

Only one trade allowed per day
Hello World
